This is a notebook for:
1. Webscrap for cities data
2. Save data into cities DataFrame
3. Webscrap for population data
4. Save data into population DataFrame
5. Update cities and population tables with collected data

The cities DataFrame should contain:
* city_id
* population 
* timestamp_population
* longitude/latitude

The population DataFrame should contain:
* city_id
* city_name
* country_code

### 1. Import of the needed Modules, setting up and checking url, extracting data for soup:

In [15]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import sqlalchemy
import keys

Simplified solution to work faster with:

In [16]:
# An easy to use list of European cities to simplify the tasks if needed:
url_list = "https://en.wikipedia.org/wiki/List_of_European_cities_by_population_within_city_limits"
list_response = requests.get(url_list)
list_response.status_code

200

Maybe needed for later if we wanted to work with actual City pages:

In [17]:
# # The actually to be explored cities in a list that can be expanded:
# cities = ["Berlin"]
# # Base url to be expanded by city names later:
# url_base = "https://en.wikipedia.org/wiki/"
# # A loop through the city list to get the city url:
# for city in cities:
#     url_city_page = requests.get(url_base + city)
#     # print(url_city_page.status_code) # To see if it works
#     citysoup = BeautifulSoup(url_city_page.content, "html.parser") # Brewing the soup
#     # Setting up the DF to reduce to needed stuff
#     rawdata = {} 
#     rawdata["city"] = city
# # Another check if it works:
# # citysoup.select("#firstHeading")
# # citysoup.select("#firstHeading")[0].get_text() # put in comments sinc it worked out perfectly.

### 2. Html content from "url_list", reduced to "brew soup":

In [18]:
# Put in comments to enhance readability:
# list_response.content

In [19]:
# As done in the Code-Along:
soup = BeautifulSoup(list_response.content)
rows = soup.select("table.wikitable tr")
# rows # to see if it works
cells = rows[1].select("td")
# cells # to see if it works
# cells[1].get_text()
# cells[2].get_text()
# cells[3].get_text()
# for cell in cells: # to see if it works
#     print("_______________________________")
#     print(cell)

### 3. Creating the dataframe/.csv needed to work with:

In [20]:
# Setting up a dictionary:
cities_dict = {
    'city':[],
    'country':[],
    'population':[],
    'lat':[],
    'long':[]
}

In [21]:
# Loop through html to get the needed data into the df, print statements to see if it works:
for i, row in enumerate(rows):
    if(i == 0 ):
        continue
    # print("______________________________")
    # print(row.select('td')[1].select('a')[0].get_text().strip())
    cities_dict['city'].append(row.select('td')[1].select('a')[0].get_text().strip())
    # print(row.select('td')[2].select('a')[0].get_text().strip())
    cities_dict['country'].append(row.select('td')[2].select('a')[0].get_text().strip())
    # print(row.select('td')[3].select('span')[0].get_text())
    cities_dict['population'].append(row.select('td')[3].select('span')[0].get_text())
    # print(row.select('td')[7].select('span.geo-dec')[0].get_text().split()[0])
    cities_dict['lat'].append(row.select('td')[7].select('span.geo-dec')[0].get_text().split()[0])
    # print(row.select('td')[7].select('span.geo-dec')[0].get_text().split()[1])
    cities_dict['long'].append(row.select('td')[7].select('span.geo-dec')[0].get_text().split()[1])

In [22]:
city_df = pd.DataFrame(cities_dict)
# city_df # see if it works


In [23]:
# Just for changing lat and long from string to float:
city_df[['lat_','temp']] = city_df['lat'].str.split('°',expand=True)
city_df['lat_'] = pd.to_numeric(city_df['lat_'], errors='coerce')
city_df['lat'] = city_df.apply(lambda x: x['lat_'] * (-1) if x['temp'] == 'S' else x['lat_'], axis=1)
city_df[['long_','temp2']] = city_df['long'].str.split('°',expand=True)
city_df['long_'] = pd.to_numeric(city_df['long_'], errors='coerce')
city_df['long'] = city_df.apply(lambda x: x['long_'] * (-1) if x['temp2'] == 'W' else x['long_'], axis=1)
city_df.drop(['lat_','temp','long_','temp2'],axis=1,inplace=True)
# Remove special characters in pop:
city_df['population'] = pd.to_numeric(city_df['population'].str.replace(',',''))

In [24]:
# city_df.to_csv('city.csv',index=False)

In [25]:
# print(city_df) # to see how it works

### 4. After setting up "gans":

In [26]:
host = '127.0.0.1'
schema = 'gans'
user = 'root'
password = keys.mysql_PW
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [27]:
# To not make 30 API calls with one run, we reduce the df to x cities:
x_cities = ['Budapest', 'Prague']
city_df = city_df.loc[city_df['city'].isin(x_cities)]
print(city_df)
city_df.to_csv('city.csv',index=False)

        city         country  population        lat       long
15  Budapest         Hungary     1706851  47.492500  19.051389
23    Prague  Czech Republic     1275406  50.083333  14.416667


In [28]:
city_df.to_sql('city',con=con,if_exists='append',index=False)

2